In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [7]:
val sameModel = PipelineModel.load(modelPath)
val predictionsDF = sameModel.transform(inputDF)
val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

sameModel = pipeline_df775b066cd9
predictionsDF = [tweet: string, words: array<string> ... 6 more fields]
getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [8]:
predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
    batchDF.select(getProbability($"probability").alias("clean_probability")).show
}.start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@50eeeeda

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+
|               tweet|               words|            features|     indexedFeatures|       rawPrediction|         probability|prediction|predictedLabel|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------+
|@DeFrankYouth psh...|[@defrankyouth, p...|(1000,[45,170,213...|(1000,[45,170,213...|[4.96883415040153...|[0.49688341504015...|       1.0|             1|
|Is it corny to cr...|[is, it, corny, t...|(1000,[281,329,35...|(1000,[281,329,35...|[5.02262508165549...|[0.50226250816554...|       0.0|             0|
|@EmilyMakar awwww...|[@emilymakar, aww...|(1000,[71,200,329...|(1000,[71,200,329...|[5.29019003190587...|[0.52901900319058...|       0.0|             0|
|titanic makes me ...|[titanic, makes, ...|(1000,[170,330,34...|(1000,[170,3

In [ ]:

predictionsDF.writeStream.foreachBatch { (batchDF: DataFrame, batchId: Long) =>
    batchDF.show()
}.start()